# **CellTracksColab**
---

<font size = 4>Colab Notebook for compiling and analyzing Tracks


<font size = 4>Notebook created by [Guillaume Jacquemet](https://cellmig.org/)



# **Part 0. Before getting started**
---

<font size = 5>**Important notes**

---
## Data Requirements for Analysis

<font size = 4>Be advised of one significant limitation inherent to this notebook.

<font size = 4 color="red">**This notebook does not support Track splitting**</font>. <font size = 4>For users aiming to compute additional track metrics within this environment, it is crucial to disable track splitting in TrackMate.

<font size = 4>It’s important to clarify that the absence of track splitting support does not hinder the notebook's ability to compile and display results in part 3 of the analysis process.

---

## Folder Hierarchy

<font size = 4>To load your TrackMate outputs, your dataset should be meticulously organized into a two-tiered folder hierarchy as depicted below. We also expect that your track files ends by `-tracks` and that your spot tables ends by `-spots`.

<font size = 4>Here's a common data structure that can work:

- 📁 **Experiments** `[Folder_path]`
  - 🌿 **Condition_1** `[‘condition’ is derived from this folder name]`
    - 🔄 **R1** `[‘repeat’ is derived from this folder name]`
      - 📄 `FOV1-spots.csv`
      - 📄 `FOV1-tracks.csv`
      - 📄 `FOV2-spots.csv`
      - 📄 `FOV2-tracks.csv`
    - 🔄 **R2**
      - 📄 `FOV1-spots.csv`
      - 📄 `FOV1-tracks.csv`
      - 📄 `FOV2-spots.csv`
      - 📄 `FOV2-tracks.csv`
  - 🌿 **Condition_2**
    - 🔄 **R1**
    - 🔄 **R2**

<font size = 4>In this representation, different symbols are used to represent folders and files clearly:

📁 represents the main folder or directory.
🌿 represents the condition folders.
🔄 represents the repeat folders.
📄 represents the individual CSV files.

---

## Test dataset

A test dataset can be downloaded directly in this notebook or is available here:

https://zenodo.org/record/8413510

---


In [ ]:
# @title #MIT License

print("""
**MIT License**

Copyright (c) 2023 Guillaume Jacquemet

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.""")

--------------------------------------------------------
# **Part 0. Prepare the Google Colab session (skip this section when using a local installation)**
--------------------------------------------------------


## **0.1. Install key dependencies**
---
<font size = 4>

In [ ]:
#@markdown ##Play to install
!git clone --single-branch --branch dev-egm https://github.com/CellMigrationLab/CellTracksColab.git
!pip install -r "CellTracksColab/requirements.txt"

## **0.2. Mount your Google Drive**
---
<font size = 4> To use this notebook on the data present in your Google Drive, you need to mount your Google Drive to this notebook.

<font size = 4> Play the cell below to mount your Google Drive and follow the instructions.

<font size = 4> Once this is done, your data are available in the **Files** tab on the top left of notebook.

In [ ]:
#@markdown ##Play the cell to connect your Google Drive to Colab
from google.colab import drive
drive.mount('/content/drive')
# This command was originally but I think it doesn't do anything really
## %cd /gdrive

--------------------------------------------------------
# **Part 1. Prepare the session and load the data**
--------------------------------------------------------

## **1.1 Load key dependencies**
---
<font size = 4>

In [ ]:
#@markdown ##Play to load the dependancies
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import itertools
from matplotlib.gridspec import GridSpec
import requests
import ipywidgets as widgets
from ipywidgets import interact
from tqdm.notebook import tqdm

# Current version of the notebook the user is running
current_version = "1.0.1"
Notebook_name = 'Main'

# URL to the raw content of the version file in the repository
version_url = "https://raw.githubusercontent.com/guijacquemet/CellTracksColab/main/Notebook/latest_version.txt"

# Function to define colors for formatting messages
class bcolors:
    WARNING = '\033[91m'  # Red color for warning messages
    ENDC = '\033[0m'      # Reset color to default

# Check if this is the latest version of the notebook
try:
    All_notebook_versions = pd.read_csv(version_url, dtype=str)
    print('Notebook version: ' + current_version)

    # Check if 'Version' column exists in the DataFrame
    if 'Version' in All_notebook_versions.columns:
        Latest_Notebook_version = All_notebook_versions[All_notebook_versions["Notebook"] == Notebook_name]['Version'].iloc[0]
        print('Latest notebook version: ' + Latest_Notebook_version)

        if current_version == Latest_Notebook_version:
            print("This notebook is up-to-date.")
        else:
            print(bcolors.WARNING + "A new version of this notebook has been released. We recommend that you download it at https://github.com/guijacquemet/CellTracksColab" + bcolors.ENDC)
    else:
        print("The 'Version' column is not present in the version file.")
except requests.exceptions.RequestException as e:
    print("Unable to fetch the latest version information. Please check your internet connection.")
except Exception as e:
    print("An error occurred:", str(e))

#----------------------- Key functions -----------------------------#

# Function to calculate Cohen's d
def cohen_d(group1, group2):
    diff = group1.mean() - group2.mean()
    n1, n2 = len(group1), len(group2)
    var1 = group1.var()
    var2 = group2.var()
    pooled_var = ((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2)
    d = diff / np.sqrt(pooled_var)
    return d

def save_dataframe_with_progress(df, path, desc="Saving", chunk_size=50000):
    """Save a DataFrame with a progress bar."""

    # Estimating the number of chunks based on the provided chunk size
    num_chunks = int(len(df) / chunk_size) + 1

    # Create a tqdm instance for progress tracking
    with tqdm(total=len(df), unit="rows", desc=desc) as pbar:
        # Open the file for writing
        with open(path, "w") as f:
            # Write the header once at the beginning
            df.head(0).to_csv(f, index=False)

            for chunk in np.array_split(df, num_chunks):
                chunk.to_csv(f, mode="a", header=False, index=False)
                pbar.update(len(chunk))

def check_for_nans(df, df_name):
    """
    Checks the given DataFrame for NaN values and prints the count for each column containing NaNs.

    Args:
    df (pd.DataFrame): DataFrame to be checked for NaN values.
    df_name (str): The name of the DataFrame as a string, used for printing.
    """
    # Check if the DataFrame has any NaN values and print a warning if it does.
    nan_columns = df.columns[df.isna().any()].tolist()

    if nan_columns:
        for col in nan_columns:
            nan_count = df[col].isna().sum()
            print(f"Column '{col}' in {df_name} contains {nan_count} NaN values.")
    else:
        print(f"No NaN values found in {df_name}.")


import pandas as pd
import os

def save_parameters(params, file_path, param_type):
    # Convert params dictionary to a DataFrame for human readability
    new_params_df = pd.DataFrame(list(params.items()), columns=['Parameter', 'Value'])
    new_params_df['Type'] = param_type

    if os.path.exists(file_path):
        # Read existing file
        existing_params_df = pd.read_csv(file_path)

        # Merge the new parameters with the existing ones
        # Update existing parameters or append new ones
        updated_params_df = pd.merge(existing_params_df, new_params_df,
                                     on=['Type', 'Parameter'],
                                     how='outer',
                                     suffixes=('', '_new'))

        # If there's a new value, update it, otherwise keep the old value
        updated_params_df['Value'] = updated_params_df['Value_new'].combine_first(updated_params_df['Value'])

        # Drop the temporary new value column
        updated_params_df.drop(columns='Value_new', inplace=True)
    else:
        # Use new parameters DataFrame directly if file doesn't exist
        updated_params_df = new_params_df

    # Save the updated DataFrame to CSV
    updated_params_df.to_csv(file_path, index=False)




## **1.2. Compile your data or load existing dataframes**
---

<font size = 4> Please ensure that your data is properly organised (see above)


In [ ]:
## This section is the code is to test the same functions in all notebooks.
## Once they work, we can copy paste all the code in the Part 1. Load Key dependencies.
import sys
# The following paths may vary a bit locally dependending on where the jupyter lab is running.
# It's basically the path to the github repository, also in colab
sys.path.append("../")
sys.path.append("CellTracksColab/")
import celltracks

In [ ]:
#@markdown ##Provide the path to your dataset:

#@markdown ###You have multiple tracking files you want to compile, provide the path to your:

Folder_path = ''  # @param {type: "string"}
Data_Dims = "2D" #@param ["2D", "3D"]
File_Format = "xml" #@param ["csv", "xml"]
Data_Type = "TrackMate Files"

#@markdown ###Or use a test dataset (up to 10 min download)
Use_test_dataset = False #@param {type:"boolean"}

#@markdown ###Provide the path to your Result folder

Results_Folder = ""  # @param {type: "string"}

# Update the parameters to load the data
CellTracks = celltracks.TrackingData()
if Use_test_dataset:
    # Download the test dataset
    CellTracks.DownloadTestData()
CellTracks.Folder_path = Folder_path

CellTracks.Results_Folder = Results_Folder
CellTracks.skiprows = None
CellTracks.data_type = Data_Type
CellTracks.data_dims = Data_Dims
CellTracks.fileformat = File_Format

# Load data
CellTracks.LoadTrackingData()

merged_spots_df = CellTracks.spots_data
check_for_nans(merged_spots_df, "merged_spots_df")
merged_tracks_df = CellTracks.tracks_data

print("Data columns have been mapped as follows (Needed column: column name in the data):")
CellTracks.dim_mapping

print("...Done")

## **1.4. Spatial and Temporal Calibration for your data (optional)**
---
<font size = 4>If your tracking data are currently in pixel units and you wish to recalibrate them to reflect actual physical units (e.g., micrometers), you can use the provided widget to input your calibration values.

<font size = 5> **Instructions**:

1. **Spatial Calibration**:
    - **X Calibration**: Input the real-world length (in your desired unit, e.g., micrometers) corresponding to a single pixel's width in your images.
    - **Y Calibration**: Input the real-world length (in your desired unit) corresponding to a single pixel's height in your images.
    - **Z Calibration**: Input the real-world depth (in your desired unit) corresponding to a single pixel's depth in z-stacked images. If you have a 2D dataset, leave to 1.
    - **Spatial Unit**: Specify the unit of measurement for your spatial calibration (e.g., "micron", "mm").

2. **Temporal Calibration**:
    - **Temporal Calibration**: Input the real-world time (in your desired time unit, e.g., seconds) corresponding to the interval between frames in your tracking data.
    - **Time Unit**: Specify the unit of measurement for your temporal calibration (e.g., "second", "minute").

3. After inputting your calibration values, click the **"Save Calibration"** button. This action will recalibrate the `POSITION_X`, `POSITION_Y`, `POSITION_Z`, and `POSITION_T` columns of your `merged_spots_df` DataFrame to reflect the real-world units.

<font size = 5> **Note**:
    - The test dataset is already calibrated.


In [ ]:
# @title ##Run the cell to calibrate your data

#@markdown ### Do you need to
calibrate_units = False #@param {type:"boolean"}

#@markdown ### If True, what are the spatial calibration values?
x_cal = 1. #@param {type:"number"}
y_cal = 1. #@param {type:"number"}
#@markdown ### Only if the data is 3D:
z_cal = 1. #@param {type:"number"}
#@markdown ### and what are the spatial calibration units?
spatial_calibration_unit = "pixel" # @param {type: "string"}

#@markdown ### If True, what are the temporal calibration values?
t_cal = 1. #@param {type:"number"}
#@markdown ### and what are the temporal calibration units?
time_unit = "second" # @param {type: "string"}


if calibrate_units:
    # Recalibrate the dimensions
    CellTracks.CalibrateUnits(self, x_cal, y_cal, z_cal, t_cal, spatial_calibration_unit, time_unit)
    # Load the data updated
    merged_spots_df = CellTracks.spots_data

## **1.5. Visualise your tracks**
---

### **1.5.1 Visualise your tracks in each field of view**
---

In [ ]:
# @title ##Run the cell and choose the file you want to inspect
import os
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt

if not os.path.exists(os.path.join(Results_Folder, "Tracks")):
    os.makedirs(os.path.join(Results_Folder, "Tracks"), exist_ok=True)  # Create Results_Folder if it doesn't exist

# Extract unique filenames from the dataframe
filenames = merged_spots_df['File_name'].unique()

# Create a Dropdown widget with the filenames
filename_dropdown = widgets.Dropdown(
    options=filenames,
    value=filenames[0] if len(filenames) > 0 else None,  # Default selected value
    description='File Name:',
)

def plot_coordinates(filename, display_plots=True):
    if filename:
        # Filter the DataFrame based on the selected filename
        filtered_df = merged_spots_df[merged_spots_df['File_name'] == filename]

        plt.figure(figsize=(10, 8))
        for unique_id in filtered_df['Unique_ID'].unique():
            unique_df = filtered_df[filtered_df['Unique_ID'] == unique_id].sort_values(by='POSITION_T')
            plt.plot(unique_df['POSITION_X'], unique_df['POSITION_Y'], marker='o', linestyle='-', markersize=2)

        plt.xlabel('POSITION_X')
        plt.ylabel('POSITION_Y')
        plt.title(f'Coordinates for {filename}')
        plt.savefig(f"{Results_Folder}/Tracks/Tracks_{filename}.pdf")
        if display_plots:  # Only display plots if explicitly requested
          plt.show()
        else:
          plt.close()  # Close the plot to prevent it from displaying
    else:
        print("No valid filename selected")

# Link the Dropdown widget to the plotting function
interact(plot_coordinates, filename=filename_dropdown)


In [ ]:
import os
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# @title ##Process all field of view

display_plots = False # @param {type:"boolean"}

def plot_coordinates(filename, display_plots=True):
    if filename:
        # Filter the DataFrame based on the selected filename
        filtered_df = merged_spots_df[merged_spots_df['File_name'] == filename]

        plt.figure(figsize=(10, 8))
        for unique_id in filtered_df['Unique_ID'].unique():
            unique_df = filtered_df[filtered_df['Unique_ID'] == unique_id].sort_values(by='POSITION_T')
            plt.plot(unique_df['POSITION_X'], unique_df['POSITION_Y'], marker='o', linestyle='-', markersize=2)

        plt.xlabel('POSITION_X')
        plt.ylabel('POSITION_Y')
        plt.title(f'Coordinates for {filename}')
        plt.savefig(f"{Results_Folder}/Tracks/Tracks_{filename}.pdf")
        if display_plots:  # Only display plots if explicitly requested
          plt.show()
        else:
          plt.close()  # Close the plot to prevent it from displaying
    else:
        print("No valid filename selected")


# Adjusted function to plot all FOVs with tqdm progress bar and saving message
def plot_all_fovs():
    print("Plotting and saving tracks for all FOVs...")
    for filename in tqdm(filenames, desc="Processing"):
        plot_coordinates(filename, display_plots=display_plots)
    print(f"All plots saved in: {Results_Folder}/Tracks/")

plot_all_fovs()



### **1.5.2 Origin-Normalized Plot for each field of view**

In [ ]:
# @title ##Run the cell and choose the file you want to inspect

display_plots=True

import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import os

if not os.path.exists(os.path.join(Results_Folder, "Tracks")):
    os.makedirs(os.path.join(Results_Folder, "Tracks"), exist_ok=True)  # Create Results_Folder if it doesn't exist

# Extract unique filenames from the dataframe
filenames = merged_spots_df['File_name'].unique()

# Create a Dropdown widget with the filenames
filename_dropdown = widgets.Dropdown(
    options=filenames,
    value=filenames[0] if len(filenames) > 0 else None,  # Default selected value
    description='File Name:',
)

def plot_normalized_coordinates(filename, display_plots=True):
    if filename:
        # Filter the DataFrame based on the selected filename
        filtered_df = merged_spots_df[merged_spots_df['File_name'] == filename]

        plt.figure(figsize=(10, 8))

        # Group by Unique_ID to work with each track individually
        for unique_id in filtered_df['Unique_ID'].unique():
            unique_df = filtered_df[filtered_df['Unique_ID'] == unique_id].sort_values(by='POSITION_T')

            # Normalize starting point to (0, 0)
            start_x = unique_df.iloc[0]['POSITION_X']
            start_y = unique_df.iloc[0]['POSITION_Y']
            normalized_x = unique_df['POSITION_X'] - start_x
            normalized_y = unique_df['POSITION_Y'] - start_y

            # Plot the normalized track without adding to the legend
            plt.plot(normalized_x, normalized_y, marker='o', linestyle='-', markersize=2)

        plt.xlabel('Normalized POSITION_X')
        plt.ylabel('Normalized POSITION_Y')
        plt.title(f'Origin-Normalized Tracks for {filename}')
        plt.savefig(f"{Results_Folder}/Tracks/Origin_Normalized_Tracks_{filename}.pdf")
        if display_plots:  # Only display plots if explicitly requested
          plt.show()
        else:
          plt.close()  # Close the plot to prevent it from displaying
    else:
        print("No valid filename selected")

# Link the Dropdown widget to the plotting function
interact(plot_normalized_coordinates, filename=filename_dropdown)


In [ ]:
import os
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# @title ##Process all field of view

display_plots = False # @param {type:"boolean"}



def plot_normalized_coordinates(filename, display_plots=True):
    if filename:
        # Filter the DataFrame based on the selected filename
        filtered_df = merged_spots_df[merged_spots_df['File_name'] == filename]

        plt.figure(figsize=(10, 8))

        # Group by Unique_ID to work with each track individually
        for unique_id in filtered_df['Unique_ID'].unique():
            unique_df = filtered_df[filtered_df['Unique_ID'] == unique_id].sort_values(by='POSITION_T')

            # Normalize starting point to (0, 0)
            start_x = unique_df.iloc[0]['POSITION_X']
            start_y = unique_df.iloc[0]['POSITION_Y']
            normalized_x = unique_df['POSITION_X'] - start_x
            normalized_y = unique_df['POSITION_Y'] - start_y

            # Plot the normalized track without adding to the legend
            plt.plot(normalized_x, normalized_y, marker='o', linestyle='-', markersize=2)

        plt.xlabel('Normalized POSITION_X')
        plt.ylabel('Normalized POSITION_Y')
        plt.title(f'Origin-Normalized Tracks for {filename}')
        plt.savefig(f"{Results_Folder}/Tracks/Origin_Normalized_Tracks_{filename}.pdf")
        if display_plots:  # Only display plots if explicitly requested
          plt.show()
        else:
          plt.close()  # Close the plot to prevent it from displaying
    else:
        print("No valid filename selected")


# Adjusted function to plot all FOVs with tqdm progress bar and saving message
def plot_all_fovs():
    print("Plotting and saving tracks for all FOVs...")
    for filename in tqdm(filenames, desc="Processing"):
        plot_normalized_coordinates(filename, display_plots=display_plots)
    print(f"All plots saved in: {Results_Folder}/Tracks/")

plot_all_fovs()



### **1.5.3 Origin-Normalized Plot for each condition and repeat**

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import os

# @title ##Run the cell and choose the file you want to inspect


if not os.path.exists(os.path.join(Results_Folder, "Tracks")):
    os.makedirs(os.path.join(Results_Folder, "Tracks"), exist_ok=True)  # Ensure the directory exists for saving the plots

# Assuming your DataFrame has 'Condition' and 'Repeat' columns
conditions = merged_spots_df['Condition'].unique()
repeats = merged_spots_df['Repeat'].unique()

condition_dropdown = widgets.Dropdown(
    options=conditions,
    value=conditions[0] if len(conditions) > 0 else None,
    description='Condition:',
)

repeat_dropdown = widgets.Dropdown(
    options=repeats,
    value=repeats[0] if len(repeats) > 0 else None,
    description='Repeat:',
)

def plot_normalized_coordinates(condition, repeat, display_plots=True):
    # Filter the DataFrame based on the selected condition and repeat
    filtered_df = merged_spots_df[(merged_spots_df['Condition'] == condition) &
                                  (merged_spots_df['Repeat'] == repeat)]

    if not filtered_df.empty:
        plt.figure(figsize=(10, 8))

        # Iterate over each unique track ID in the filtered DataFrame
        for unique_id in filtered_df['Unique_ID'].unique():
            track_df = filtered_df[filtered_df['Unique_ID'] == unique_id].sort_values(by='POSITION_T')

            # Normalize starting point to (0, 0)
            start_x = track_df.iloc[0]['POSITION_X']
            start_y = track_df.iloc[0]['POSITION_Y']
            normalized_x = track_df['POSITION_X'] - start_x
            normalized_y = track_df['POSITION_Y'] - start_y

            # Plot the normalized track
            plt.plot(normalized_x, normalized_y, marker='o', linestyle='-', markersize=2)

        plt.xlabel('Normalized POSITION_X')
        plt.ylabel('Normalized POSITION_Y')
        plt.title(f'Origin-Normalized Tracks for Condition: {condition}, Repeat: {repeat}')

        if display_plots:
            plt.show()
        else:
            plt.close()

        # Optionally save the plot
        plot_filename = f"Condition_{condition}_Repeat_{repeat}.pdf"
        plt.savefig(os.path.join(Results_Folder, "Tracks", plot_filename))
        print(f"Plot saved as {plot_filename} in {Results_Folder}/Tracks/")
    else:
        print("No data available for the selected condition and repeat.")

# Link the widgets to the plotting function
interact(plot_normalized_coordinates, condition=condition_dropdown, repeat=repeat_dropdown)

In [ ]:
# @title ##Process all Repeat/Condition combinations


display_plots = False # @param {type:"boolean"}

if not os.path.exists(os.path.join(Results_Folder, "Tracks")):
  os.makedirs(os.path.join(Results_Folder, "Tracks"), exist_ok=True)

def plot_normalized_coordinates(condition, repeat, display_plots=True):
    # Filter the DataFrame based on the selected condition and repeat
    filtered_df = merged_spots_df[(merged_spots_df['Condition'] == condition) &
                                  (merged_spots_df['Repeat'] == repeat)]

    if not filtered_df.empty:
        plt.figure(figsize=(10, 8))

        # Iterate over each unique track ID in the filtered DataFrame
        for unique_id in filtered_df['Unique_ID'].unique():
            track_df = filtered_df[filtered_df['Unique_ID'] == unique_id].sort_values(by='POSITION_T')

            # Normalize starting point to (0, 0)
            start_x = track_df.iloc[0]['POSITION_X']
            start_y = track_df.iloc[0]['POSITION_Y']
            normalized_x = track_df['POSITION_X'] - start_x
            normalized_y = track_df['POSITION_Y'] - start_y

            # Plot the normalized track
            plt.plot(normalized_x, normalized_y, marker='o', linestyle='-', markersize=2)

        plt.xlabel('Normalized POSITION_X')
        plt.ylabel('Normalized POSITION_Y')
        plt.title(f'Origin-Normalized Tracks for Condition: {condition}, Repeat: {repeat}')

        if display_plots:
            plt.show()
        else:
            plt.close()

        # Optionally save the plot
        plot_filename = f"Condition_{condition}_Repeat_{repeat}.pdf"
        plt.savefig(os.path.join(Results_Folder, "Tracks", plot_filename))
        print(f"Plot saved as {plot_filename} in {Results_Folder}/Tracks/")
    else:
        print("No data available for the selected condition and repeat.")


# Adjusted function to plot all FOVs with tqdm progress bar and saving message
def plot_all_fovs(display_plots=False):

    conditions = merged_spots_df['Condition'].unique()
    repeats = merged_spots_df['Repeat'].unique()

    print("Plotting and saving tracks for all combinations of Conditions and Repeats...")

    # Iterate over all combinations of Condition and Repeat
    for condition in tqdm(conditions, desc="Conditions"):
        for repeat in tqdm(repeats, desc="Repeats", leave=False):  # Nested progress bar for repeats
            plot_normalized_coordinates(condition, repeat, display_plots=display_plots)

    print(f"All plots saved in: {Results_Folder}/Tracks/")

# Call the function with display_plots set to your preference
plot_all_fovs(display_plots=display_plots)

### **1.5.4 Origin-Normalized Plot for each condition**

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import os

# @title ##Run the cell and choose the file you want to inspect


if not os.path.exists(os.path.join(Results_Folder, "Tracks")):
  os.makedirs(os.path.join(Results_Folder, "Tracks"), exist_ok=True) # Ensure the directory exists for saving the plots

# Assuming your DataFrame has 'Condition' and 'Repeat' columns
conditions = merged_spots_df['Condition'].unique()


condition_dropdown = widgets.Dropdown(
    options=conditions,
    value=conditions[0] if len(conditions) > 0 else None,
    description='Condition:',
)


def plot_normalized_coordinates(condition, display_plots=True):
    # Filter the DataFrame based on the selected condition and repeat
    filtered_df = merged_spots_df[(merged_spots_df['Condition'] == condition)]

    if not filtered_df.empty:
        plt.figure(figsize=(10, 8))

        # Iterate over each unique track ID in the filtered DataFrame
        for unique_id in filtered_df['Unique_ID'].unique():
            track_df = filtered_df[filtered_df['Unique_ID'] == unique_id].sort_values(by='POSITION_T')

            # Normalize starting point to (0, 0)
            start_x = track_df.iloc[0]['POSITION_X']
            start_y = track_df.iloc[0]['POSITION_Y']
            normalized_x = track_df['POSITION_X'] - start_x
            normalized_y = track_df['POSITION_Y'] - start_y

            # Plot the normalized track
            plt.plot(normalized_x, normalized_y, marker='o', linestyle='-', markersize=2)

        plt.xlabel('Normalized POSITION_X')
        plt.ylabel('Normalized POSITION_Y')
        plt.title(f'Origin-Normalized Tracks for Condition: {condition}')

        if display_plots:
            plt.show()
        else:
            plt.close()

        # Optionally save the plot
        plot_filename = f"Condition_{condition}_All_Repeat.pdf"
        plt.savefig(os.path.join(Results_Folder, "Tracks", plot_filename))
        print(f"Plot saved as {plot_filename} in {Results_Folder}/Tracks/")
    else:
        print("No data available for the selected condition.")

# Link the widgets to the plotting function
interact(plot_normalized_coordinates, condition=condition_dropdown)

In [ ]:
# @title ##Process all conditions

display_plots = False # @param {type:"boolean"}

if not os.path.exists(os.path.join(Results_Folder, "Tracks")):
  os.makedirs(os.path.join(Results_Folder, "Tracks"), exist_ok=True)

def plot_normalized_coordinates(condition, display_plots=True):
    # Filter the DataFrame based on the selected condition and repeat
    filtered_df = merged_spots_df[(merged_spots_df['Condition'] == condition)]

    if not filtered_df.empty:
        plt.figure(figsize=(10, 8))

        # Iterate over each unique track ID in the filtered DataFrame
        for unique_id in filtered_df['Unique_ID'].unique():
            track_df = filtered_df[filtered_df['Unique_ID'] == unique_id].sort_values(by='POSITION_T')

            # Normalize starting point to (0, 0)
            start_x = track_df.iloc[0]['POSITION_X']
            start_y = track_df.iloc[0]['POSITION_Y']
            normalized_x = track_df['POSITION_X'] - start_x
            normalized_y = track_df['POSITION_Y'] - start_y

            # Plot the normalized track
            plt.plot(normalized_x, normalized_y, marker='o', linestyle='-', markersize=2)

        plt.xlabel('Normalized POSITION_X')
        plt.ylabel('Normalized POSITION_Y')
        plt.title(f'Origin-Normalized Tracks for Condition: {condition}')

        if display_plots:
            plt.show()
        else:
            plt.close()

        # Optionally save the plot
        plot_filename = f"Condition_{condition}_All_Repeat.pdf"
        plt.savefig(os.path.join(Results_Folder, "Tracks", plot_filename))
        print(f"Plot saved as {plot_filename} in {Results_Folder}/Tracks/")
    else:
        print("No data available for the selected condition.")


# Adjusted function to plot all FOVs with tqdm progress bar and saving message
def plot_all_fovs(display_plots=False):

    conditions = merged_spots_df['Condition'].unique()

    print("Plotting and saving tracks for all Conditions...")

    # Iterate over all combinations of Condition
    for condition in tqdm(conditions, desc="Conditions"):
      plot_normalized_coordinates(condition, display_plots=display_plots)

    print(f"All plots saved in: {Results_Folder}/Tracks/")

# Call the function with display_plots set to your preference
plot_all_fovs(display_plots=display_plots)

--------------------------------------------------------
# **Part 2. Compute track metrics**
--------------------------------------------------------
<font size = 4 color="red">Part2 does not support Track splitting</font>.



<font size = 4>The following section provides an interactive way to refine your tracking data. Here's what it's designed to achieve:

1. <font size = 4>**Filter Tracks**:
    - <font size = 4>This feature allows you to define a range for the track lengths you're interested in. By adjusting the `Min Length` and `Max Length` sliders, you can ignore very short or very long tracks that might be artifacts or noise in your data.

2. <font size = 4>**Smooth Tracks**:
    - <font size = 4>The positional data in your tracks can be smoothed using a moving average technique. By adjusting the `Smoothing` slider, you can control the degree of smoothing applied to the tracks. A higher value will average over more points, producing smoother tracks. This can be beneficial if your raw data has a lot of jitter or minor positional fluctuations.

<font size = 4>**How to Use**:

- <font size = 4>**Min Length**: Use the slider to set the minimum length of the tracks you're interested in.
- <font size = 4>**Max Length**: Use the slider to set the maximum length of the tracks you're interested in.
- <font size = 4>**Smoothing**: Adjust this slider to control the degree of smoothing you'd like to apply to your tracks.
- <font size = 4>**Apply Filters**: After adjusting the sliders to your preference, click this button. This will process the data based on your choices and prepare it for downstream analyses.



## **2.1. Filter and smooth your tracks (Optional)**
---

In [ ]:
from tqdm.notebook import tqdm

# @title ##Run to filter and smooth your tracks (slow when the dataset is large)

import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import numpy as np

def save_filter_smoothing_params(file_path, min_length, max_length, smoothing_neighbors):
    params = {
        'Min Length': min_length,
        'Max Length': max_length,
        'Smoothing Neighbors': smoothing_neighbors
    }
    params_df = pd.DataFrame([params])
    params_df.to_csv(file_path, index=False)

def optimized_filter_and_smooth_tracks(dataframe, min_length=5, max_length=100, smoothing_neighbors=3):
    track_lengths = dataframe.groupby('Unique_ID').size()
    valid_tracks = track_lengths[(track_lengths >= min_length) & (track_lengths <= max_length)].index
    filtered_df = dataframe[dataframe['Unique_ID'].isin(valid_tracks)]

    def smooth_track(track_data):
        track_data = track_data.sort_values(by='POSITION_T')
        smoothed_track = track_data.copy()

        if len(track_data) >= smoothing_neighbors:
            smoothed_X = track_data['POSITION_X'].rolling(window=smoothing_neighbors, center=True).mean()
            smoothed_Y = track_data['POSITION_Y'].rolling(window=smoothing_neighbors, center=True).mean()
            smoothed_Z = track_data['POSITION_Z'].rolling(window=smoothing_neighbors, center=True).mean()

            smoothed_track['POSITION_X'] = smoothed_X
            smoothed_track['POSITION_Y'] = smoothed_Y
            smoothed_track['POSITION_Z'] = smoothed_Z

        return smoothed_track

    # Use tqdm for progress bar
    tqdm.pandas(desc="Processing Tracks")
    smoothed_df = filtered_df.groupby('Unique_ID').progress_apply(smooth_track).reset_index(drop=True)

    return smoothed_df


def on_button_click(button):
    global filtered_and_smoothed_df
    with output:
        clear_output(wait=True)

        filtered_and_smoothed_df = optimized_filter_and_smooth_tracks(
            merged_spots_df,
            min_length=min_length_slider.value,
            max_length=max_length_slider.value,
            smoothing_neighbors=smoothing_slider.value
        )

        save_dataframe_with_progress(filtered_and_smoothed_df, os.path.join(Results_Folder, 'filtered_and_smoothed_Spots.csv'))

        # Save the parameters
        params_file_path = os.path.join(Results_Folder, "filter_smoothing_parameters.csv")
        save_filter_smoothing_params(params_file_path, min_length_slider.value, max_length_slider.value, smoothing_slider.value)

        print("Done")




# Calculate the maximum track length from the data
max_track_length = merged_spots_df.groupby('Unique_ID').size().max()

# Define widgets for user input
min_length_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=max_track_length,
    step=1,
    description='Min Length:',
    continuous_update=False
)

max_length_slider = widgets.IntSlider(
    value=max_track_length,
    min=1,
    max=max_track_length,
    step=1,
    description='Max Length:',
    continuous_update=False
)

smoothing_slider = widgets.IntSlider(
    value=3,
    min=1,
    max=10,
    step=1,
    description='Smoothing:',
    continuous_update=False
)


apply_button = widgets.Button(description="Apply Filters", button_style='info')
output = widgets.Output()

apply_button.on_click(on_button_click)

# Display the widgets
display(widgets.VBox([min_length_slider, max_length_slider, smoothing_slider, apply_button, output]))




In [ ]:
# @title ##Compare Raw vs Filtered tracks

from ipywidgets import interactive
from IPython.display import display
from IPython.display import clear_output

import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt

if not os.path.exists(Results_Folder+"/Tracks"):
    os.makedirs(Results_Folder+"/Tracks")  # Create Results_Folder if it doesn't exist

# Extract unique filenames from the dataframe
filenames = merged_spots_df['File_name'].unique()

# Create a Dropdown widget with the filenames
filename_dropdown = widgets.Dropdown(
    options=filenames,
    value=filenames[0] if len(filenames) > 0 else None,  # Default selected value
    description='File Name:',
)


def plot_coordinates_side_by_side(filename):
    if filename:
        # Filter the DataFrames based on the selected filename
        raw_df = merged_spots_df[merged_spots_df['File_name'] == filename]
        processed_df = filtered_and_smoothed_df[filtered_and_smoothed_df['File_name'] == filename]

        # Create subplots
        fig, axes = plt.subplots(1, 2, figsize=(20, 8))

        # Create a colormap to ensure consistent colors across tracks
        unique_ids = raw_df['Unique_ID'].unique()
        colormap = plt.get_cmap('tab20')

        # Plot Raw Data
        for idx, unique_id in enumerate(unique_ids):
            unique_data = raw_df[raw_df['Unique_ID'] == unique_id].sort_values(by='POSITION_T')
            color_val = colormap(idx % 20 / 20)  # Ensure colors are within colormap range
            axes[0].plot(unique_data['POSITION_X'], unique_data['POSITION_Y'],
                         color=color_val, marker='o', linestyle='-', markersize=2)
        axes[0].set_title(f'Raw Coordinates for {filename}')
        axes[0].set_xlabel('POSITION_X')
        axes[0].set_ylabel('POSITION_Y')

        # Plot Filtered & Smoothed Data
        for idx, unique_id in enumerate(unique_ids):
            unique_data = processed_df[processed_df['Unique_ID'] == unique_id].sort_values(by='POSITION_T')
            color_val = colormap(idx % 20 / 20)  # Ensure colors are within colormap range
            axes[1].plot(unique_data['POSITION_X'], unique_data['POSITION_Y'],
                         color=color_val, marker='o', linestyle='-', markersize=2)
        axes[1].set_title(f'Filtered & Smoothed Coordinates for {filename}')
        axes[1].set_xlabel('POSITION_X')
        axes[1].set_ylabel('POSITION_Y')

        plt.tight_layout()
        plt.savefig(f"{Results_Folder}/Tracks/Filtered_tracks_{filename}.pdf")
        plt.show()
    else:
        print("No valid filename selected")


def on_filename_change(change):
    # Clear the previous plots
    clear_output(wait=True)

    # Redisplay the dropdown
    display(filename_dropdown)

    # Generate and display the new plots
    plot_coordinates_side_by_side(change['new'])

# Define a button-click callback
def on_plot_button_click(button):
    # Clear previous plots
    clear_output(wait=True)

    # Redisplay the dropdown and button
    display(filename_dropdown)
    display(plot_button)

    # Generate and display the new plots
    plot_coordinates_side_by_side(filename_dropdown.value)

# Create the plot button and attach the callback
plot_button = widgets.Button(
    description="Plot",
    button_style='info'
)
plot_button.on_click(on_plot_button_click)

# Initial display of the dropdown widget and the plot button
display(filename_dropdown)
display(plot_button)



In [ ]:
# @title ##Run to choose which data you want to use for further analysis

import ipywidgets as widgets

# Layout for the widgets
widget_layout = widgets.Layout(width='500px')

# Create a RadioButtons widget to allow users to choose the DataFrame
data_choice = widgets.RadioButtons(
    options=[('Raw data', 'raw'), ('Smooth and filtered data', 'smoothed')],
    description='Use:',
    value='raw',
    disabled=False,
    layout=widget_layout
)

# Create a global variable to store the chosen dataframe
spots_df_to_use = None

# Create a button for analysis
analyze_button = widgets.Button(
    description="Analyze",
    button_style='info',
    layout=widget_layout
)

# Define the button click callback
def on_analyze_button_click(button):
    global spots_df_to_use

    # Check the selected value and update the global dataframe accordingly
    if data_choice.value == 'raw':
        spots_df_to_use = merged_spots_df
    elif data_choice.value == 'smoothed':
        spots_df_to_use = filtered_and_smoothed_df
    else:
        print("Invalid choice!")
        return

    print(f"Analysis will be performed using: {data_choice.label}")


# Assign button callback
analyze_button.on_click(on_analyze_button_click)

# Initial display of the widgets
display(data_choice)
display(analyze_button)


## **2.2. Duration and speed metrics**
---
<font size = 4>When this cell is executed, it calculates various metrics for each unique track. Specifically, for each track, it determines the duration of the track, the average, maximum, minimum, and standard deviation of speeds, as well as the total distance travelled by the tracked object.

<font size = 4> *TrackMate data has these parameters already calculated so this cell could be skipped.*

In [ ]:
# @title ##Calculate track metrics

from tqdm.notebook import tqdm

print("Calculating track metrics...")

# Check if spots_df_to_use is None or empty; if so, set it to merged_spots_df
if 'spots_df_to_use' not in globals() or spots_df_to_use is None or spots_df_to_use.empty:
    spots_df_to_use = merged_spots_df

spots_df_to_use.dropna(subset=['POSITION_X', 'POSITION_Y', 'POSITION_Z'], inplace=True)

def calculate_track_metrics(group):
    group = group.sort_values('POSITION_T')
    positions = group[['POSITION_X', 'POSITION_Y', 'POSITION_Z']].values
    times = group['POSITION_T'].values

    # Track Duration
    duration = times[-1] - times[0]

    # Speeds calculation
    deltas = np.linalg.norm(np.diff(positions, axis=0), axis=1)
    time_diffs = np.diff(times)
    time_diffs[time_diffs == 0] = 1e-10
    speeds = deltas / time_diffs

    return pd.Series({
        'Track Duration': duration,
        'Mean Speed': speeds.mean(),
        'Max Speed': speeds.max(),
        'Min Speed': speeds.min(),
        'Speed Standard Deviation': speeds.std(),
        'Total Distance Traveled': deltas.sum()
    })

tqdm.pandas(desc="Calculating Track Metrics")

# Sort and then calculate metrics
spots_df_to_use.sort_values(by=['Unique_ID', 'POSITION_T'], inplace=True)
df_track_metrics = spots_df_to_use.groupby('Unique_ID').progress_apply(calculate_track_metrics).reset_index()

# Find overlapping columns, drop them, then merge new metrics into merged_tracks_df
overlapping_columns = merged_tracks_df.columns.intersection(df_track_metrics.columns).drop('Unique_ID')
merged_tracks_df.drop(columns=overlapping_columns, inplace=True)
merged_tracks_df = pd.merge(merged_tracks_df, df_track_metrics, on='Unique_ID', how='left')

# Save the DataFrame (assuming you have a function called save_dataframe_with_progress)
save_dataframe_with_progress(merged_tracks_df, os.path.join(Results_Folder, 'merged_Tracks.csv'))

# Safeguard: check for NaN
check_for_nans(merged_tracks_df, "merged_tracks_df")
#check_for_nans(merged_spots_df, "merged_spots_df")

print("...Done")


## **2.3. Directionality**
---
<font size = 4>To calculate the directionality of a track in 3D space, we consider a series of points each with \(x\), \(y\), and \(z\) coordinates, sorted by time. The directionality, denoted as \(D\), is calculated using the formula:

$$ D = \frac{d_{\text{euclidean}}}{d_{\text{total path}}} $$

where \($d_{\text{euclidean}}$\) is the Euclidean distance between the first and the last points of the track, calculated as:

$$ d_{\text{euclidean}} = \sqrt{(x_{\text{end}} - x_{\text{start}})^2 + (y_{\text{end}} - y_{\text{start}})^2 + (z_{\text{end}} - z_{\text{start}})^2} $$

and \($d_{\text{total path}}$\) is the sum of the Euclidean distances between all consecutive points in the track, representing the total path length traveled. If the total path length is zero, the directionality is defined to be zero. This measure provides insight into the straightness of the path taken, with a value of 1 indicating a straight path between the start and end points, and values approaching 0 indicating more circuitous paths.</font>


In [ ]:
# @title ##Calculate directionality
import pandas as pd
import numpy as np

print("In progress...")

# Check if spots_df_to_use is None or empty; if so, set it to merged_spots_df
if 'spots_df_to_use' not in globals() or spots_df_to_use is None or spots_df_to_use.empty:
    spots_df_to_use = merged_spots_df

spots_df_to_use.dropna(subset=['POSITION_X', 'POSITION_Y', 'POSITION_Z'], inplace=True)

# Function to calculate Directionality
def calculate_directionality(group):

    group = group.sort_values('POSITION_T')
    start_point = group.iloc[0][['POSITION_X', 'POSITION_Y', 'POSITION_Z']].to_numpy()
    end_point = group.iloc[-1][['POSITION_X', 'POSITION_Y', 'POSITION_Z']].to_numpy()

    # Calculating Euclidean distance in 3D between start and end points
    euclidean_distance = np.linalg.norm(end_point - start_point)

    # Calculating the total path length in 3D
    deltas = np.linalg.norm(np.diff(group[['POSITION_X', 'POSITION_Y', 'POSITION_Z']].values, axis=0), axis=1)
    total_path_length = deltas.sum()

    # Calculating Directionality
    D = euclidean_distance / total_path_length if total_path_length != 0 else 0

    return pd.Series({'Directionality': D})


# Create a tqdm object for the groupby apply
tqdm.pandas(desc="Calculating Directionality")

# Assuming spots_df_to_use is your DataFrame
spots_df_to_use.sort_values(by=['Unique_ID', 'POSITION_T'], inplace=True)

# Calculate directionality for each track
df_directionality = spots_df_to_use.groupby('Unique_ID').progress_apply(calculate_directionality).reset_index()

# Find the overlapping columns between the two DataFrames, excluding the merging key
overlapping_columns = merged_tracks_df.columns.intersection(df_directionality.columns).drop('Unique_ID')

# Drop the overlapping columns from the left DataFrame
merged_tracks_df.drop(columns=overlapping_columns, inplace=True)

# Merge the directionality back into the original DataFrame
merged_tracks_df = pd.merge(merged_tracks_df, df_directionality, on='Unique_ID', how='left')

# Save the DataFrame with the calculated directionality
save_dataframe_with_progress(merged_tracks_df, os.path.join(Results_Folder, 'merged_Tracks.csv'))

# Safeguard: check for NaN
check_for_nans(merged_tracks_df, "merged_tracks_df")
#check_for_nans(merged_spots_df, "merged_spots_df")

print("...Done")


## **2.4. Tortuosity**
---
<font size = 4>This measure provides insight into the curvature and complexity of the path taken, with a value of 1 indicating a straight path between the start and end points, and values greater than 1 indicating paths with more twists and turns.
To calculate the tortuosity of a track in 3D space, we consider a series of points each with \(x\), \(y\), and \(z\) coordinates, sorted by time. The tortuosity, denoted as \(T\), is calculated using the formula:

$$ T = \frac{d_{\text{total path}}}{d_{\text{euclidean}}} $$



In [ ]:
# @title ##Calculate tortuosity

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

print("In progress...")

# Initialize tqdm with pandas
tqdm.pandas(desc="Calculating Tortuosity")

# Check if spots_df_to_use is None or empty; if so, set it to merged_spots_df
if 'spots_df_to_use' not in globals() or spots_df_to_use is None or spots_df_to_use.empty:
    spots_df_to_use = merged_spots_df

def calculate_tortuosity(group):
    group = group.sort_values('POSITION_T')

    # Apply spatial calibration to the coordinates
    calibrated_coords = group[['POSITION_X', 'POSITION_Y', 'POSITION_Z']].values

    start_point = calibrated_coords[0]
    end_point = calibrated_coords[-1]

    # Calculating Euclidean distance in 3D between start and end points
    euclidean_distance = np.linalg.norm(end_point - start_point)

    # Calculating the total path length in 3D
    deltas = np.linalg.norm(np.diff(calibrated_coords, axis=0), axis=1)
    total_path_length = deltas.sum()

    # Calculating Tortuosity
    T = total_path_length / euclidean_distance if euclidean_distance != 0 else 0

    return pd.Series({'Tortuosity': T})

# Sort the DataFrame by 'Unique_ID' and 'POSITION_T'
spots_df_to_use.sort_values(by=['Unique_ID', 'POSITION_T'], inplace=True)

# Calculate tortuosity for each track using progress_apply
df_tortuosity = spots_df_to_use.groupby('Unique_ID').progress_apply(calculate_tortuosity).reset_index()

# Find the overlapping columns between the two DataFrames, excluding the merging key
overlapping_columns = merged_tracks_df.columns.intersection(df_tortuosity.columns).drop('Unique_ID')

# Drop the overlapping columns from the left DataFrame
merged_tracks_df.drop(columns=overlapping_columns, inplace=True)

# Merge the tortuosity back into the original DataFrame
merged_tracks_df = pd.merge(merged_tracks_df, df_tortuosity, on='Unique_ID', how='left')

# Save the DataFrame with the calculated tortuosity
save_dataframe_with_progress(merged_tracks_df, Results_Folder + '/' + 'merged_Tracks.csv')

# Safeguard: check for NaN
check_for_nans(merged_tracks_df, "merged_tracks_df")
#check_for_nans(merged_spots_df, "merged_spots_df")

print("...Done")


## **2.5. Calculate the total turning angle**
---

<font size = 4>This measure provides insight into the cumulative amount of turning along the path, with a value of 0 indicating a straight path with no turning, and higher values indicating paths with more turning.

<font size = 4>To calculate the Total Turning Angle of a track in 3D space, we consider a series of points each with \(x\), \(y\), and \(z\) coordinates, sorted by time. The Total Turning Angle, denoted as \(A\), is the sum of the angles between each pair of consecutive direction vectors along the track, representing the cumulative amount of turning along the path.

<font size = 4>For each pair of consecutive segments in the track, we calculate the direction vectors \( $\vec{v_1}$ \) and \($ \vec{v_2}$ \), and the angle \($ \theta$ \) between them is calculated using the formula:

$$ \cos(\theta) = \frac{\vec{v_1} \cdot \vec{v_2}}{||\vec{v_1}|| \cdot ||\vec{v_2}||} $$

<font size = 4>where \( $\vec{v_1} \cdot$ $\vec{v_2}$ \) is the dot product of the direction vectors, and \( $||\vec{v_1}||$ \) and \( $||\vec{v_2}||$ \) are the magnitudes of the direction vectors. The Total Turning Angle \( $A$ \) is then the sum of all the angles \( \$theta$ \) calculated between each pair of consecutive direction vectors along the track:

$$ A = \sum \theta $$
<font size = 4>
If either of the direction vectors is a zero vector, the angle between them is undefined, and such cases are skipped in the calculation.


In [ ]:
# @title ##Calculate the total turning angle

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

print("In progress...")

# Check if spots_df_to_use is None or empty; if so, set it to merged_spots_df
if 'spots_df_to_use' not in globals() or spots_df_to_use is None or spots_df_to_use.empty:
    spots_df_to_use = merged_spots_df

# Initialize tqdm with pandas
tqdm.pandas(desc="Calculating Total Turning Angle")

# Check if spots_df_to_use is None or empty; if so, set it to merged_spots_df
if 'spots_df_to_use' not in globals() or spots_df_to_use is None or spots_df_to_use.empty:
    spots_df_to_use = merged_spots_df

def calculate_total_turning_angle(group):
    group = group.sort_values('POSITION_T')
    directions = group[['POSITION_X', 'POSITION_Y', 'POSITION_Z']].diff().dropna()
    total_turning_angle = 0

    for i in range(1, len(directions)):
        dir1 = directions.iloc[i - 1]
        dir2 = directions.iloc[i]

        if np.linalg.norm(dir1) == 0 or np.linalg.norm(dir2) == 0:
            continue

        cos_angle = np.dot(dir1, dir2) / (np.linalg.norm(dir1) * np.linalg.norm(dir2))
        cos_angle = np.clip(cos_angle, -1, 1)
        angle = np.degrees(np.arccos(cos_angle))
        total_turning_angle += angle

    return pd.Series({'Total_Turning_Angle': total_turning_angle})

# Sort the DataFrame by 'Unique_ID' and 'POSITION_T'
spots_df_to_use.sort_values(by=['Unique_ID', 'POSITION_T'], inplace=True)

# Calculate total turning angle for each track using progress_apply instead of apply
df_turning_angle = spots_df_to_use.groupby('Unique_ID').progress_apply(calculate_total_turning_angle).reset_index()

# Check if 'Total_Turning_Angle' is in the columns of df_turning_angle
if 'Total_Turning_Angle' not in df_turning_angle.columns:
    print("Error: 'Total_Turning_Angle' not in df_turning_angle columns")

# Find the overlapping columns between the two DataFrames, excluding the merging key
overlapping_columns = merged_tracks_df.columns.intersection(df_turning_angle.columns).drop('Unique_ID')

# Drop the overlapping columns from the left DataFrame
merged_tracks_df.drop(columns=overlapping_columns, inplace=True)

# Merge the total turning angle back into the original DataFrame
merged_tracks_df = pd.merge(merged_tracks_df, df_turning_angle, on='Unique_ID', how='left')

# Save the DataFrame with the calculated total turning angle
save_dataframe_with_progress(merged_tracks_df, os.path.join(Results_Folder, 'merged_Tracks.csv'))

# Safeguard: check for NaN
check_for_nans(merged_tracks_df, "merged_tracks_df")
#check_for_nans(merged_spots_df, "merged_spots_df")

print("...Done")


## **2.6. Calculate the Spatial Coverage**
---

<font size = 4>Spatial coverage provides insight into the spatial extent covered by the object's movement, with higher values indicating that the object has covered a larger area or volume during its movement.


<font size = 4>To calculate the spatial coverage of a track in 2D or 3D space, we consider a series of points each with \(x\), \(y\), and optionally \(z\) coordinates, sorted by time. The spatial coverage, denoted as \(S\), represents the area (in 2D) or volume (in 3D) enclosed by the convex hull formed by the points in the track. It provides insight into the spatial extent covered by the moving object.

<font size = 4> **In the implementation below we:**
1. <font size = 4>**Check Dimensionality**:
   <font size = 4>- If the variance of the \(z\) coordinates is zero, implying all \(z\) coordinates are the same, the spatial coverage is calculated in 2D using only the \(x\) and \(y\) coordinates.
  <font size = 4> - If the \(z\) coordinates vary, the spatial coverage is calculated in 3D using the \(x\), \(y\), and \(z\) coordinates.

2. <font size = 4>**Form Convex Hull**:
   <font size = 4>- In 2D, a minimum of 3 non-collinear points is required to form a convex hull.
   <font size = 4>- In 3D, a minimum of 4 non-coplanar points is required to form a convex hull.
   <font size = 4>- If the required minimum points are not available, the spatial coverage is defined to be zero.

3. <font size = 4>**Calculate Spatial Coverage**:
   <font size = 4>- In 2D, the spatial coverage \(S\) is the area of the convex hull formed by the points in the track.
   <font size = 4>- In 3D, the spatial coverage \(S\) is the volume of the convex hull formed by the points in the track.

<font size = 4> **Formula:**
- For 2D Spatial Coverage (Area of Convex Hull), if points are \(P_1(x_1, y_1), P_2(x_2, y_2), \ldots, P_n(x_n, y_n)\):
  $$ S_{2D} = \text{Area of Convex Hull formed by } P_1, P_2, \ldots, P_n $$

- For 3D Spatial Coverage (Volume of Convex Hull), if points are \(P_1(x_1, y_1, z_1), P_2(x_2, y_2, z_2), \ldots, P_n(x_n, y_n, z_n)\):
  $$ S_{3D} = \text{Volume of Convex Hull formed by } P_1, P_2, \ldots, P_n $$



In [ ]:
# @title ##Calculate the Spatial Coverage

import pandas as pd
import numpy as np
from scipy.spatial import ConvexHull
from tqdm.notebook import tqdm

print("In progress...")

# Initialize tqdm with pandas
tqdm.pandas(desc="Calculating Spatial Coverage")

# Check if spots_df_to_use is None or empty; if so, set it to merged_spots_df
if 'spots_df_to_use' not in globals() or spots_df_to_use is None or spots_df_to_use.empty:
    spots_df_to_use = merged_spots_df

def calculate_spatial_coverage_with_calibration(group):
    group = group.sort_values('POSITION_T')

    # Apply spatial calibration to the coordinates
    calibrated_coords = group[['POSITION_X', 'POSITION_Y', 'POSITION_Z']].values  # Ensure .values is added here

    # Drop rows with NaN values in the coordinates
    calibrated_coords = calibrated_coords[~np.isnan(calibrated_coords).any(axis=1)]

    # Check the variance of Z coordinates
    z_variance = np.var(calibrated_coords[:, 2])

    if z_variance == 0:  # If variance of Z is 0, calculate 2D spatial coverage
        if len(calibrated_coords) < 3:  # Need at least 3 points for a 2D convex hull
            return pd.Series({'Spatial_Coverage': 0})

        try:
            coords_2d = calibrated_coords[:, :2]  # Use only X and Y coordinates
            hull_2d = ConvexHull(coords_2d, qhull_options='QJ')  # 'QJ' joggles the input to avoid precision errors
            spatial_coverage = hull_2d.volume  # Area of the convex hull in 2D
        except Exception as e:
            print(f"Error calculating 2D spatial coverage: {e}")
            spatial_coverage = 0
    else:  # If variance of Z is not 0, calculate 3D spatial coverage
        if len(calibrated_coords) < 4:  # Need at least 4 points for a 3D convex hull
            return pd.Series({'Spatial_Coverage': 0})

        try:
            hull = ConvexHull(calibrated_coords, qhull_options='QJ')  # 'QJ' joggles the input to avoid precision errors
            spatial_coverage = hull.volume  # Volume of the convex hull in 3D
        except Exception as e:
            print(f"Error calculating 3D spatial coverage: {e}")
            spatial_coverage = 0

    return pd.Series({'Spatial_Coverage': spatial_coverage})


# Sort the DataFrame by 'Unique_ID' and 'POSITION_T'
spots_df_to_use.sort_values(by=['Unique_ID', 'POSITION_T'], inplace=True)

# Calculate spatial coverage for each track using progress_apply instead of apply
df_spatial_coverage = spots_df_to_use.groupby('Unique_ID').progress_apply(calculate_spatial_coverage_with_calibration).reset_index()

# Find the overlapping columns between the two DataFrames, excluding the merging key
overlapping_columns = merged_tracks_df.columns.intersection(df_spatial_coverage.columns).drop('Unique_ID')

# Drop the overlapping columns from the left DataFrame
merged_tracks_df.drop(columns=overlapping_columns, inplace=True)

# Merge the spatial coverage back into the original DataFrame
merged_tracks_df = pd.merge(merged_tracks_df, df_spatial_coverage, on='Unique_ID', how='left')

# Save the DataFrame with the calculated spatial coverage
save_dataframe_with_progress(merged_tracks_df, os.path.join(Results_Folder, 'merged_Tracks.csv'))

# Safeguard: check for NaN
check_for_nans(merged_tracks_df, "merged_tracks_df")
#check_for_nans(merged_spots_df, "merged_spots_df")

print("...Done")


--------
# **Part 3. Quality Control**
--------

      



## **3.1. Assess if your dataset is balanced**
---

In cell tracking and similar biological analyses, the balance of the dataset is important, particularly in ensuring that each biological repeat carries equal weight. Here's why this balance is essential:

### Accurate Representation of Biological Variability

- **Capturing True Biological Variation**: Biological repeats are crucial for capturing the natural variability inherent in biological systems. Equal weighting ensures that this variability is accurately represented.
- **Reducing Sampling Bias**: By balancing the dataset, we avoid overemphasizing the characteristics of any single repeat, which might not be representative of the broader biological context.

If your data is too imbalanced, it may be useful to ensure that this does not shift your results.



In [ ]:
import pandas as pd

# @title ##Check the number of track per condition per repeats


if not os.path.exists(f"{Results_Folder}/QC"):
    os.makedirs(f"{Results_Folder}/QC")


import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import os

def count_tracks_by_condition_and_repeat(df, Results_Folder, condition_col='Condition', repeat_col='Repeat', track_id_col='Unique_ID'):
    """
    Counts the number of unique tracks for each combination of condition and repeat in the given DataFrame and
    saves a stacked histogram plot as a PDF in the QC folder with annotations for each stack.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the data.
    Results_Folder (str): The base folder where the results will be saved.
    condition_col (str): The name of the column representing the condition. Default is 'Condition'.
    repeat_col (str): The name of the column representing the repeat. Default is 'Repeat'.
    track_id_col (str): The name of the column representing the track ID. Default is 'Unique_ID'.
    """
    track_counts = df.groupby([condition_col, repeat_col])[track_id_col].nunique()
    track_counts_df = track_counts.reset_index()
    track_counts_df.rename(columns={track_id_col: 'Number_of_Tracks'}, inplace=True)

    # Pivot the data for plotting
    pivot_df = track_counts_df.pivot(index=condition_col, columns=repeat_col, values='Number_of_Tracks').fillna(0)

    # Plotting
    fig, ax = plt.subplots(figsize=(12, 6))
    bars = pivot_df.plot(kind='bar', stacked=True, ax=ax)
    ax.set_xlabel('Condition')
    ax.set_ylabel('Number of Tracks')
    ax.set_title('Stacked Histogram of Track Counts per Condition and Repeat')
    ax.legend(title=repeat_col)
    ax.grid(axis='y', linestyle='--')

    # Hide horizontal grid lines
    ax.yaxis.grid(False)

    # Add number annotations on each stack
    for bar in bars.patches:
        ax.text(bar.get_x() + bar.get_width() / 2,
                bar.get_y() + bar.get_height() / 2,
                int(bar.get_height()),
                ha='center', va='center', color='black', fontweight='bold', fontsize=8)

    # Save the plot as a PDF
    pdf_file = os.path.join(Results_Folder, 'Track_Counts_Histogram.pdf')
    plt.savefig(pdf_file, bbox_inches='tight')
    print(f"Saved histogram to {pdf_file}")

    plt.show()

    return track_counts_df

result_df = count_tracks_by_condition_and_repeat(merged_tracks_df, f"{Results_Folder}/QC")




## **3.2. Compute Similarity Metrics between Field of Views (FOV) and between Conditions and Repeats**
---

<font size = 4>**Purpose**:

<font size = 4>This section provides a set of tools to compute and visualize similarities between different field of views (FOV) based on selected track parameters. By leveraging hierarchical clustering, the resulting dendrogram offers a clear visualization of how different FOV, conditions, or repeats relate to one another. This tool is essential for:

<font size = 4>1. **Quality Control**:
    - Ensuring that FOVs from the same condition or experimental setup are more similar to each other than to FOVs from different conditions.
    - Confirming that repeats of the same experiment yield consistent results and cluster together.
    
<font size = 4>2. **Data Integrity**:
    - Identifying potential outliers or anomalies in the dataset.
    - Assessing the overall consistency of the experiment and ensuring reproducibility.

<font size = 4>**How to Use**:

<font size = 4>1. **Track Parameters Selection**:
    - A list of checkboxes allows users to select which track parameters they want to consider for similarity calculations. By default, all parameters are selected. Users can deselect parameters that they believe might not contribute significantly to the similarity.

<font size = 4>2. **Similarity Metric**:
    - Users can choose a similarity metric from a dropdown list. Options include cosine, euclidean, cityblock, jaccard, and correlation. The choice of similarity metric can influence the clustering results, so users might need to experiment with different metrics to see which one provides the most meaningful results.

<font size = 4>3. **Linkage Method**:
    - Determines how the distance between clusters is calculated in the hierarchical clustering process. Different linkage methods can produce different dendrograms, so users might want to try various methods.

<font size = 4>4. **Visualization**:
    - Once the parameters are selected, users can click on the "Select the track parameters and visualize similarity" button. This will compute the hierarchical clustering and display two dendrograms:
        - One dendrogram displays similarities between individual FOVs.
        - Another dendrogram aggregates the data based on conditions and repeats, providing a higher-level view of the similarities.
      


In [ ]:
# @title ##Compute similarity metrics between FOV and between conditions and repeats

import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import ipywidgets as widgets
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import pdist

# Check and create "QC" folder
if not os.path.exists(os.path.join(Results_Folder, "QC")):
    os.makedirs(os.path.join(Results_Folder, "QC"), exist_ok=True)

# Columns to exclude
excluded_columns = ['Condition', 'experiment_nb', 'File_name', 'Repeat', 'Unique_ID', 'LABEL', 'TRACK_INDEX', 'TRACK_ID', 'TRACK_X_LOCATION', 'TRACK_Y_LOCATION', 'TRACK_Z_LOCATION', 'Exemplar','TRACK_STOP', 'TRACK_START', 'Cluster_UMAP', 'Cluster_tsne']

selected_df = pd.DataFrame()

# Filter out non-numeric columns but keep 'File_name'
numeric_df = merged_tracks_df.select_dtypes(include=['float64', 'int64']).copy()
numeric_df['File_name'] = merged_tracks_df['File_name']

# Create a list of column names excluding 'File_name'
column_names = [col for col in numeric_df.columns if col not in excluded_columns]

# Create a checkbox for each column
checkboxes = [widgets.Checkbox(value=True, description=col, indent=False) for col in column_names]

# Dropdown for similarity metrics
similarity_dropdown = widgets.Dropdown(
    options=['cosine', 'euclidean', 'cityblock', 'jaccard', 'correlation'],
    value='cosine',
    description='Similarity Metric:'
)

# Dropdown for linkage methods
linkage_dropdown = widgets.Dropdown(
    options=['single', 'complete', 'average', 'ward'],
    value='single',
    description='Linkage Method:'
)

# Arrange checkboxes in a 2x grid
grid = widgets.GridBox(checkboxes, layout=widgets.Layout(grid_template_columns="repeat(2, 300px)"))

# Create a button to trigger the selection and visualization
button = widgets.Button(description="Select the track parameters and visualize similarity", layout=widgets.Layout(width='400px'), button_style='info')

# Define the button click event handler
def on_button_click(b):
    global selected_df  # Declare selected_df as global

    # Get the selected columns from the checkboxes
    selected_columns = [box.description for box in checkboxes if box.value]
    selected_columns.append('File_name')  # Always include 'File_name'

    # Extract the selected columns from the DataFrame
    selected_df = numeric_df[selected_columns]

    # Check and print the percentage of NaNs for each selected column
    for column in selected_columns:
        if selected_df[column].isna().any():
            nan_percentage = selected_df[column].isna().mean() * 100
            print("Warning: NaN values found in the selected data.")
            print(f"{column}: {nan_percentage:.2f}%")
            any_nan = True
            print("Proceeding to handle NaN values.")
            selected_df = selected_df.dropna()


    # Aggregate the data by filename
    aggregated_by_filename = selected_df.groupby('File_name').mean(numeric_only=True)

    # Aggregate the data by condition and repeat
    aggregated_by_condition_repeat = merged_tracks_df.groupby(['Condition', 'Repeat'])[selected_columns].mean(numeric_only=True)

    # Compute condensed distance matrices
    distance_matrix_filename = pdist(aggregated_by_filename, metric=similarity_dropdown.value)
    distance_matrix_condition_repeat = pdist(aggregated_by_condition_repeat, metric=similarity_dropdown.value)

    # Perform hierarchical clustering
    linked_filename = linkage(distance_matrix_filename, method=linkage_dropdown.value)
    linked_condition_repeat = linkage(distance_matrix_condition_repeat, method=linkage_dropdown.value)

    annotation_text = f"Similarity Method: {similarity_dropdown.value}, Linkage Method: {linkage_dropdown.value}"

        # Prepare the parameters dictionary
    similarity_params = {
        'Similarity Metric': similarity_dropdown.value,
        'Linkage Method': linkage_dropdown.value,
        'Selected Columns': ', '.join(selected_columns)
    }

    # Save the parameters
    params_file_path = os.path.join(Results_Folder, "QC", "analysis_parameters.csv")
    save_parameters(similarity_params, params_file_path, 'Similarity Metrics')

    # Plot the dendrograms one under the other
    plt.figure(figsize=(10, 10))

    # Dendrogram for individual filenames
    plt.subplot(2, 1, 1)
    dendrogram(linked_filename, labels=aggregated_by_filename.index, orientation='top', distance_sort='descending', leaf_rotation=90)
    plt.title(f'Dendrogram of Field of view Similarities\n{annotation_text}')

    # Dendrogram for aggregated data based on condition and repeat
    plt.subplot(2, 1, 2)
    dendrogram(linked_condition_repeat, labels=aggregated_by_condition_repeat.index, orientation='top', distance_sort='descending', leaf_rotation=90)
    plt.title(f'Dendrogram of Aggregated Similarities by Condition and Repeat\n{annotation_text}')

    plt.tight_layout()

    # Save the dendrogram to a PDF
    pdf_pages = PdfPages(os.path.join(Results_Folder, "QC", "Dendrogram_Similarities.pdf"))

    # Save the current figure to the PDF
    pdf_pages.savefig()

    # Close the PdfPages object to finalize the document
    pdf_pages.close()

    plt.show()

# Set the button click event handler
button.on_click(on_button_click)

# Display the widgets
display(grid, similarity_dropdown, linkage_dropdown, button)


-------------------------------------------

# **Part 4. Plot track parameters**
-------------------------------------------

<font size = 4> In this section you can plot all the track parameters previously computed. Data and graphs are automatically saved in your result folder.

<b>Note on Units:</b> The parameters plotted are in the unit of measurement you used when tracking your data.
</font>

<font size="4" color="red">
<b>Results Storage:</b>
Results generated by in this section are saved  in the sub-folder named `track_parameters_plots` within your `Results_Folder`.


##**Statistical analyses**
### Cohen's d (Effect Size):
<font size = 4>Cohen's d measures the size of the difference between two groups, normalized by their pooled standard deviation. Values can be interpreted as small (0 to 0.2), medium (0.2 to 0.5), or large (0.5 and above) effects. It helps quantify how significant the observed difference is, beyond just being statistically significant.

### Randomization Test:
<font size = 4>This non-parametric test evaluates if observed differences between conditions could have arisen by random chance. It shuffles condition labels multiple times, recalculating the Cohen's d each time. The resulting p-value, which indicates the likelihood of observing the actual difference by chance, provides evidence against the null hypothesis: a smaller p-value implies stronger evidence against the null.

### Bonferroni Correction:
<font size = 4>Given multiple comparisons, the Bonferroni Correction adjusts significance thresholds to mitigate the risk of false positives. By dividing the standard significance level (alpha) by the number of tests, it ensures that only robust findings are considered significant. However, it's worth noting that this method can be conservative, sometimes overlooking genuine effects.

## **4.1. Plot your entire dataset**
--------

In [ ]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
from scipy.stats import zscore

# @title ##Plot track normalized track parameters based on conditions as an heatmap (entire dataset)

# Parameters to adapt in function of the notebook section
base_folder = os.path.join(Results_Folder, "track_parameters_plots")
Conditions = 'Condition'
df_to_plot = merged_tracks_df

# Check and create necessary directories
folders = ["pdf", "csv"]
for folder in folders:
    dir_path = os.path.join(base_folder, folder)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)


def get_selectable_columns(df):
    # Exclude certain columns from being plotted
    exclude_cols = ['Condition', 'experiment_nb', 'File_name', 'Repeat', 'Unique_ID', 'LABEL', 'TRACK_INDEX', 'TRACK_ID', 'TRACK_X_LOCATION', 'TRACK_Y_LOCATION', 'TRACK_Z_LOCATION', 'Exemplar','TRACK_STOP', 'TRACK_START', 'Cluster_UMAP', 'Cluster_tsne']
    # Select only numerical columns
    return [col for col in df.columns if (df[col].dtype.kind in 'biufc') and (col not in exclude_cols)]


def heatmap_comparison(df, Results_Folder, Conditions, variables_per_page=40):
    # Get all the selectable columns
    variables_to_plot = get_selectable_columns(df)

    # Drop rows where all elements are NaNs in the variables_to_plot columns
    df = df.dropna()

    # Compute median for each variable across Conditions
    median_values = df.groupby(Conditions)[variables_to_plot].median().transpose()

    # Normalize the median values using Z-score
    normalized_values = median_values.apply(zscore, axis=1)

    # Number of pages
    total_variables = len(variables_to_plot)
    num_pages = int(np.ceil(total_variables / variables_per_page))

    # Initialize an empty DataFrame to store all pages' data
    all_pages_data = pd.DataFrame()

    # Create a PDF file to save the heatmaps
    with PdfPages(os.path.join(Results_Folder, "Heatmaps_Normalized_Median_Values_by_Condition.pdf")) as pdf:
        for page in range(num_pages):
            start = page * variables_per_page
            end = min(start + variables_per_page, total_variables)
            page_data = normalized_values.iloc[start:end]

            # Append this page's data to the all_pages_data DataFrame
            all_pages_data = pd.concat([all_pages_data, page_data])

            plt.figure(figsize=(16, 10))
            sns.heatmap(page_data, cmap='coolwarm', annot=True, linewidths=.1)
            plt.title(f"Z-score Normalized Median Values of Variables by Condition (Page {page + 1})")
            plt.tight_layout()

            pdf.savefig()  # saves the current figure into a pdf page
            plt.show()
            plt.close()

    # Save all pages data to a single CSV file
    all_pages_data.to_csv(os.path.join(Results_Folder, "Normalized_Median_Values_by_Condition.csv"))

    print(f"Heatmaps saved to {Results_Folder}/Heatmaps_Normalized_Median_Values_by_Condition.pdf")
    print(f"All data saved to {Results_Folder}/Normalized_Median_Values_by_Condition.csv")

# Example usage
heatmap_comparison(merged_tracks_df, base_folder, Conditions)


In [ ]:
# @title ##Plot track parameters (entire dataset)

import ipywidgets as widgets
from ipywidgets import Layout, VBox, Button, Accordion, SelectMultiple, IntText
import pandas as pd
import os
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import FixedLocator



# Parameters to adapt in function of the notebook section
base_folder = os.path.join(Results_Folder, "track_parameters_plots")
Conditions = 'Condition'
df_to_plot = merged_tracks_df

# Check and create necessary directories
folders = ["pdf", "csv"]
for folder in folders:
    dir_path = os.path.join(base_folder, folder)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

def get_selectable_columns(df):
    # Exclude certain columns from being plotted
    exclude_cols = ['Condition', 'experiment_nb', 'File_name', 'Repeat', 'Unique_ID', 'LABEL', 'TRACK_INDEX', 'TRACK_ID', 'TRACK_X_LOCATION', 'TRACK_Y_LOCATION', 'TRACK_Z_LOCATION', 'Exemplar','TRACK_STOP', 'TRACK_START', 'Cluster_UMAP', 'Cluster_tsne']
    # Select only numerical columns
    return [col for col in df.columns if (df[col].dtype.kind in 'biufc') and (col not in exclude_cols)]


def display_variable_checkboxes(selectable_columns):
    # Create checkboxes for selectable columns
    variable_checkboxes = [widgets.Checkbox(value=False, description=col) for col in selectable_columns]

    # Display checkboxes in the notebook
    display(widgets.VBox([
        widgets.Label('Variables to Plot:'),
        widgets.GridBox(variable_checkboxes, layout=widgets.Layout(grid_template_columns="repeat(%d, 300px)" % 3)),
    ]))
    return variable_checkboxes

def create_condition_selector(df, column_name):
    conditions = df[column_name].unique()
    condition_selector = SelectMultiple(
        options=conditions,
        description='Conditions:',
        disabled=False,
        layout=Layout(width='100%')  # Adjusting the layout width
    )
    return condition_selector

def display_condition_selection(df, column_name):
    condition_selector = create_condition_selector(df, column_name)

    condition_accordion = Accordion(children=[VBox([condition_selector])])
    condition_accordion.set_title(0, 'Select Conditions')
    display(condition_accordion)
    return condition_selector


def plot_selected_vars(button, variable_checkboxes, df, Conditions, Results_Folder, condition_selector):

    plt.clf()  # Clear the current figure before creating a new plot
    print("Plotting in progress...")

  # Get selected variables
    variables_to_plot = [box.description for box in variable_checkboxes if box.value]
    n_plots = len(variables_to_plot)

    if n_plots == 0:
        print("No variables selected for plotting")
        return

  # Get selected conditions
    selected_conditions = condition_selector.value
    n_selected_conditions = len(selected_conditions)

    if n_selected_conditions == 0:
        print("No conditions selected for plotting")
        return

# Use only selected and ordered conditions
    filtered_df = df[df[Conditions].isin(selected_conditions)].copy()

# Initialize matrices to store effect sizes and p-values for each variable
    effect_size_matrices = {}
    p_value_matrices = {}
    bonferroni_matrices = {}

    unique_conditions = filtered_df[Conditions].unique().tolist()
    num_comparisons = len(unique_conditions) * (len(unique_conditions) - 1) // 2
    alpha = 0.05
    corrected_alpha = alpha / num_comparisons
    n_iterations = 1000

# Loop through each variable to plot
    for var in variables_to_plot:

      pdf_pages = PdfPages(os.path.join(Results_Folder, "pdf", f"{var}_Boxplots_and_Statistics.pdf"))
      effect_size_matrix = pd.DataFrame(index=unique_conditions, columns=unique_conditions)
      p_value_matrix = pd.DataFrame(index=unique_conditions, columns=unique_conditions)
      bonferroni_matrix = pd.DataFrame(index=unique_conditions, columns=unique_conditions)

      for cond1, cond2 in itertools.combinations(unique_conditions, 2):
        group1 = df[df[Conditions] == cond1][var]
        group2 = df[df[Conditions] == cond2][var]

        original_d = abs(cohen_d(group1, group2))
        effect_size_matrix.loc[cond1, cond2] = original_d
        effect_size_matrix.loc[cond2, cond1] = original_d  # Mirroring

        count_extreme = 0
        for i in range(n_iterations):
            combined = pd.concat([group1, group2])
            shuffled = combined.sample(frac=1, replace=False).reset_index(drop=True)
            new_group1 = shuffled[:len(group1)]
            new_group2 = shuffled[len(group1):]

            new_d = abs(cohen_d(new_group1, new_group2))
            if np.abs(new_d) >= np.abs(original_d):
                count_extreme += 1

        p_value = count_extreme / n_iterations
        p_value_matrix.loc[cond1, cond2] = p_value
        p_value_matrix.loc[cond2, cond1] = p_value  # Mirroring

        # Apply Bonferroni correction
        bonferroni_corrected_p_value = min(p_value * num_comparisons, 1.0)
        bonferroni_matrix.loc[cond1, cond2] = bonferroni_corrected_p_value
        bonferroni_matrix.loc[cond2, cond1] = bonferroni_corrected_p_value  # Mirroring

      effect_size_matrices[var] = effect_size_matrix
      p_value_matrices[var] = p_value_matrix
      bonferroni_matrices[var] = bonferroni_matrix

    # Concatenate the three matrices side-by-side
      combined_df = pd.concat(
        [
            effect_size_matrices[var].rename(columns={col: f"{col} (Effect Size)" for col in effect_size_matrices[var].columns}),
            p_value_matrices[var].rename(columns={col: f"{col} (P-Value)" for col in p_value_matrices[var].columns}),
            bonferroni_matrices[var].rename(columns={col: f"{col} (Bonferroni-corrected P-Value)" for col in bonferroni_matrices[var].columns})
        ], axis=1
    )

    # Save the combined DataFrame to a CSV file
      combined_df.to_csv(os.path.join(Results_Folder, "csv", f"{var}_statistics_combined.csv"))

    # Create a new figure
      fig = plt.figure(figsize=(16, 10))

    # Create a gridspec for 2 rows and 4 columns
      gs = GridSpec(2, 3, height_ratios=[1.5, 1])

    # Create the ax for boxplot using the gridspec
      ax_box = fig.add_subplot(gs[0, :])

    # Extract the data for this variable
      data_for_var = df[[Conditions, var, 'Repeat', 'File_name' ]]

    # Save the data_for_var to a CSV for replotting
      data_for_var.to_csv(f"{Results_Folder}/csv/{var}_boxplot_data.csv", index=False)

    # Calculate the Interquartile Range (IQR) using the 25th and 75th percentiles
      Q1 = df[var].quantile(0.25)
      Q3 = df[var].quantile(0.75)
      IQR = Q3 - Q1

    # Define bounds for the outliers
      multiplier = 10
      lower_bound = Q1 - multiplier * IQR
      upper_bound = Q3 + multiplier * IQR

    # Plotting
      sns.boxplot(x=Conditions, y=var, data=filtered_df, ax=ax_box, color='lightgray')  # Boxplot
      sns.stripplot(x=Conditions, y=var, data=filtered_df, ax=ax_box, hue='Repeat', dodge=True, jitter=True, alpha=0.2)  # Individual data points
      ax_box.set_ylim([max(min(filtered_df[var]), lower_bound), min(max(filtered_df[var]), upper_bound)])
      ax_box.set_title(f"{var}")
      ax_box.set_xlabel('Condition')
      ax_box.set_ylabel(var)
      tick_labels = ax_box.get_xticklabels()
      tick_locations = ax_box.get_xticks()
      ax_box.xaxis.set_major_locator(FixedLocator(tick_locations))
      ax_box.set_xticklabels(tick_labels, rotation=90)
      ax_box.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='Repeat')

    # Statistical Analyses and Heatmaps

    # Effect Size heatmap ax
      ax_d = fig.add_subplot(gs[1, 0])
      sns.heatmap(effect_size_matrices[var].fillna(0), annot=True, cmap="viridis", cbar=True, square=True, ax=ax_d, vmax=1)
      ax_d.set_title(f"Effect Size (Cohen's d) for {var}")

    # p-value heatmap ax
      ax_p = fig.add_subplot(gs[1, 1])
      sns.heatmap(p_value_matrices[var].fillna(1), annot=True, cmap="viridis_r", cbar=True, square=True, ax=ax_p, vmax=0.1)
      ax_p.set_title(f"Randomization Test p-value for {var}")

    # Bonferroni corrected p-value heatmap ax
      ax_bonf = fig.add_subplot(gs[1, 2])
      sns.heatmap(bonferroni_matrices[var].fillna(1), annot=True, cmap="viridis_r", cbar=True, square=True, ax=ax_bonf, vmax=0.1)
      ax_bonf.set_title(f"Bonferroni-corrected p-value for {var}")

      plt.tight_layout()
      pdf_pages.savefig(fig)

    # Close the PDF
      pdf_pages.close()

condition_selector = display_condition_selection(df_to_plot, Conditions)
selectable_columns = get_selectable_columns(df_to_plot)
variable_checkboxes = display_variable_checkboxes(selectable_columns)

button = Button(description="Plot Selected Variables", layout=Layout(width='400px'), button_style='info')
button.on_click(lambda b: plot_selected_vars(b, variable_checkboxes, df_to_plot, Conditions, base_folder, condition_selector))
display(button)

## **4.2. Plot a balanced dataset**
--------

## **4.2.1. Downsample your dataset to ensure that it is balanced**
--------

### Downsampling and Balancing Dataset

This section of the notebook is dedicated to addressing imbalances in the dataset, which is crucial for ensuring the accuracy and reliability of the analysis. The cell bellow will downsample the dataset to balance the number of tracks across different conditions and repeats. It allows for reproducibility by including a `random_seed` parameter, which is set to 42 by default but can be adjusted as needed.

All results from this section will be saved in the Balanced Dataset Directory created in your `Results_Folder`.




In [ ]:
# @title ##Run this cell to downsample and balance your dataset

random_seed = 42  # @param {type: "number"}

if not os.path.exists(os.path.join(Results_Folder, "Balanced_dataset")):
    os.makedirs(os.path.join(Results_Folder, "Balanced_dataset"), exist_ok=True)

def balance_dataset(df, condition_col='Condition', repeat_col='Repeat', track_id_col='Unique_ID', random_seed=None):
    """
    Balances the dataset by downsampling tracks for each condition and repeat combination.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the data.
    condition_col (str): The name of the column representing the condition.
    repeat_col (str): The name of the column representing the repeat.
    track_id_col (str): The name of the column representing the track ID.
    random_seed (int, optional): The seed for the random number generator. Default is None.

    Returns:
    pandas.DataFrame: A new DataFrame with balanced track counts.
    """
    # Group by condition and repeat, and find the minimum track count
    min_track_count = df.groupby([condition_col, repeat_col])[track_id_col].nunique().min()

    # Function to sample min_track_count tracks from each group
    def sample_tracks(group):
        return group.sample(n=min_track_count, random_state=random_seed)

    # Apply sampling to each group and concatenate the results
    balanced_merged_tracks_df = df.groupby([condition_col, repeat_col]).apply(sample_tracks).reset_index(drop=True)

    return balanced_merged_tracks_df

balanced_merged_tracks_df = balance_dataset(merged_tracks_df, random_seed=random_seed)
result_df = count_tracks_by_condition_and_repeat(balanced_merged_tracks_df, f"{Results_Folder}/Balanced_dataset")

check_for_nans(balanced_merged_tracks_df, "balanced_merged_tracks_df")
save_dataframe_with_progress(balanced_merged_tracks_df, os.path.join(Results_Folder, 'Balanced_dataset', 'merged_Tracks_balanced_dataset.csv'))


## **4.2.2. Check if the downsampling has affected data distribution**
--------

This section of the notebook generates a heatmap visualizing the Kolmogorov-Smirnov (KS) p-values for each numerical column in the dataset, comparing the distributions before and after downsampling. This heatmap serves as a tool for assessing the impact of downsampling on data quality, guiding decisions on whether the downsampled dataset is suitable for further analysis.

#### Purpose of the Heatmap
- **KS Test:** The KS test is used to determine if two samples are drawn from the same distribution. In this context, it compares the distribution of each numerical column in the original dataset (`merged_tracks_df`) with its counterpart in the downsampled dataset (`balanced_merged_tracks_df`).
- **P-Value Interpretation:** The p-value indicates the probability that the two samples come from the same distribution. A higher p-value suggests a greater likelihood that the distributions are similar.

#### Interpreting the Heatmap
- **Color Coding:** The heatmap uses a color gradient (from viridis) to represent the range of p-values. Darker colors indicate higher p-values.
- **P-Value Thresholds:**
  - **High P-Values (Lighter Areas):** Indicate that the downsampling process likely did not significantly alter the distribution of that numerical column for the specific condition-repeat group.
  - **Low P-Values (Darker Areas):** Suggest that the downsampling process may have affected the distribution significantly.
- **Varying P-Values:** Variations in color across different columns and rows help identify which specific numerical columns and condition-repeat groups are most affected by the downsampling.




In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp

# @title ##Check if your downsampling has affected your data distribution

def calculate_ks_p_value(df1, df2, column):
    """
    Calculate the KS p-value for a given column between two dataframes.

    Parameters:
    df1 (pandas.DataFrame): Original DataFrame.
    df2 (pandas.DataFrame): DataFrame after downsampling.
    column (str): Column name to compare.

    Returns:
    float: KS p-value.
    """
    return ks_2samp(df1[column].dropna(), df2[column].dropna())[1]

# Identify numerical columns
numerical_columns = merged_tracks_df.select_dtypes(include=['int64', 'float64']).columns

# Initialize a DataFrame to store KS p-values
ks_p_values = pd.DataFrame(columns=numerical_columns)

# Iterate over each group and numerical column
for group, group_df in merged_tracks_df.groupby(['Condition', 'Repeat']):
    group_p_values = []
    balanced_group_df = balanced_merged_tracks_df[(balanced_merged_tracks_df['Condition'] == group[0]) & (balanced_merged_tracks_df['Repeat'] == group[1])]
    for column in numerical_columns:
        p_value = calculate_ks_p_value(group_df, balanced_group_df, column)
        group_p_values.append(p_value)
    ks_p_values.loc[f'Condition: {group[0]}, Repeat: {group[1]}'] = group_p_values

# Maximum number of columns per heatmap
max_columns_per_heatmap = 20

# Total number of columns
total_columns = len(ks_p_values.columns)

# Calculate the number of heatmaps needed
num_heatmaps = -(-total_columns // max_columns_per_heatmap)  # Ceiling division

# File path for the PDF
pdf_filepath = Results_Folder+'/Balanced_dataset/p-Value Heatmap.pdf'

# Create a PDF file
with PdfPages(pdf_filepath) as pdf:
    # Loop through each subset of columns and create a heatmap
    for i in range(num_heatmaps):
        start_col = i * max_columns_per_heatmap
        end_col = min(start_col + max_columns_per_heatmap, total_columns)

        # Subset of columns for this heatmap
        subset_columns = ks_p_values.columns[start_col:end_col]

        # Create the heatmap for the subset of columns
        plt.figure(figsize=(12, 8))
        sns.heatmap(ks_p_values[subset_columns], cmap='viridis', vmax=0.5, vmin=0)
        plt.title(f'Kolmogorov-Smirnov P-Value Heatmap (Columns {start_col+1} to {end_col})')
        plt.xlabel('Numerical Columns')
        plt.ylabel('Condition-Repeat Groups')
        plt.tight_layout()

        # Save the current figure to the PDF
        pdf.savefig()
        plt.show()
        plt.close()

print(f"Saved all heatmaps to {pdf_filepath}")

# Save the p-values to a CSV file
ks_p_values.to_csv(os.path.join(Results_Folder, 'Balanced_dataset', 'ks_p_values.csv'))
print("Saved KS p-values to ks_p_values.csv")


## **4.2.3. Plot your balanced dataset**
--------

In [ ]:
# @title ##Plot track parameters (balanced dataset)

import ipywidgets as widgets
from ipywidgets import Layout, VBox, Button, Accordion, SelectMultiple, IntText
import pandas as pd
import os
from matplotlib.ticker import FixedLocator


# Parameters to adapt in function of the notebook section
base_folder = os.path.join(Results_Folder, "Balanced_dataset", "track_parameters_plots")
Conditions = 'Condition'
df_to_plot = balanced_merged_tracks_df

# Check and create necessary directories
folders = ["pdf", "csv"]
for folder in folders:
    dir_path = os.path.join(base_folder, folder)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

def get_selectable_columns(df):
    # Exclude certain columns from being plotted
    exclude_cols = ['Condition', 'experiment_nb', 'File_name', 'Repeat', 'Unique_ID', 'LABEL', 'TRACK_INDEX', 'TRACK_ID', 'TRACK_X_LOCATION', 'TRACK_Y_LOCATION', 'TRACK_Z_LOCATION', 'Exemplar','TRACK_STOP', 'TRACK_START', 'Cluster_UMAP', 'Cluster_tsne']
    # Select only numerical columns
    return [col for col in df.columns if (df[col].dtype.kind in 'biufc') and (col not in exclude_cols)]


def display_variable_checkboxes(selectable_columns):
    # Create checkboxes for selectable columns
    variable_checkboxes = [widgets.Checkbox(value=False, description=col) for col in selectable_columns]

    # Display checkboxes in the notebook
    display(widgets.VBox([
        widgets.Label('Variables to Plot:'),
        widgets.GridBox(variable_checkboxes, layout=widgets.Layout(grid_template_columns="repeat(%d, 300px)" % 3)),
    ]))
    return variable_checkboxes

def create_condition_selector(df, column_name):
    conditions = df[column_name].unique()
    condition_selector = SelectMultiple(
        options=conditions,
        description='Conditions:',
        disabled=False,
        layout=Layout(width='100%')  # Adjusting the layout width
    )
    return condition_selector

def display_condition_selection(df, column_name):
    condition_selector = create_condition_selector(df, column_name)

    condition_accordion = Accordion(children=[VBox([condition_selector])])
    condition_accordion.set_title(0, 'Select Conditions')
    display(condition_accordion)
    return condition_selector


def plot_selected_vars(button, variable_checkboxes, df, Conditions, Results_Folder, condition_selector):

    plt.clf()  # Clear the current figure before creating a new plot
    print("Plotting in progress...")

  # Get selected variables
    variables_to_plot = [box.description for box in variable_checkboxes if box.value]
    n_plots = len(variables_to_plot)

    if n_plots == 0:
        print("No variables selected for plotting")
        return

  # Get selected conditions
    selected_conditions = condition_selector.value
    n_selected_conditions = len(selected_conditions)

    if n_selected_conditions == 0:
        print("No conditions selected for plotting")
        return

# Use only selected and ordered conditions
    filtered_df = df[df[Conditions].isin(selected_conditions)].copy()

# Initialize matrices to store effect sizes and p-values for each variable
    effect_size_matrices = {}
    p_value_matrices = {}
    bonferroni_matrices = {}

    unique_conditions = filtered_df[Conditions].unique().tolist()
    num_comparisons = len(unique_conditions) * (len(unique_conditions) - 1) // 2
    alpha = 0.05
    corrected_alpha = alpha / num_comparisons
    n_iterations = 1000

# Loop through each variable to plot
    for var in variables_to_plot:

      pdf_pages = PdfPages(os.path.join(Results_Folder, "pdf", f"{var}_Boxplots_and_Statistics.pdf"))
      effect_size_matrix = pd.DataFrame(index=unique_conditions, columns=unique_conditions)
      p_value_matrix = pd.DataFrame(index=unique_conditions, columns=unique_conditions)
      bonferroni_matrix = pd.DataFrame(index=unique_conditions, columns=unique_conditions)

      for cond1, cond2 in itertools.combinations(unique_conditions, 2):
        group1 = df[df[Conditions] == cond1][var]
        group2 = df[df[Conditions] == cond2][var]

        original_d = abs(cohen_d(group1, group2))
        effect_size_matrix.loc[cond1, cond2] = original_d
        effect_size_matrix.loc[cond2, cond1] = original_d  # Mirroring

        count_extreme = 0
        for i in range(n_iterations):
            combined = pd.concat([group1, group2])
            shuffled = combined.sample(frac=1, replace=False).reset_index(drop=True)
            new_group1 = shuffled[:len(group1)]
            new_group2 = shuffled[len(group1):]

            new_d = abs(cohen_d(new_group1, new_group2))
            if np.abs(new_d) >= np.abs(original_d):
                count_extreme += 1

        p_value = count_extreme / n_iterations
        p_value_matrix.loc[cond1, cond2] = p_value
        p_value_matrix.loc[cond2, cond1] = p_value  # Mirroring

        # Apply Bonferroni correction
        bonferroni_corrected_p_value = min(p_value * num_comparisons, 1.0)
        bonferroni_matrix.loc[cond1, cond2] = bonferroni_corrected_p_value
        bonferroni_matrix.loc[cond2, cond1] = bonferroni_corrected_p_value  # Mirroring

      effect_size_matrices[var] = effect_size_matrix
      p_value_matrices[var] = p_value_matrix
      bonferroni_matrices[var] = bonferroni_matrix

    # Concatenate the three matrices side-by-side
      combined_df = pd.concat(
        [
            effect_size_matrices[var].rename(columns={col: f"{col} (Effect Size)" for col in effect_size_matrices[var].columns}),
            p_value_matrices[var].rename(columns={col: f"{col} (P-Value)" for col in p_value_matrices[var].columns}),
            bonferroni_matrices[var].rename(columns={col: f"{col} (Bonferroni-corrected P-Value)" for col in bonferroni_matrices[var].columns})
        ], axis=1
    )

    # Save the combined DataFrame to a CSV file
      combined_df.to_csv(os.path.join(Results_Folder, "csv", f"{var}_statistics_combined.csv"))

    # Create a new figure
      fig = plt.figure(figsize=(16, 10))

    # Create a gridspec for 2 rows and 4 columns
      gs = GridSpec(2, 3, height_ratios=[1.5, 1])

    # Create the ax for boxplot using the gridspec
      ax_box = fig.add_subplot(gs[0, :])

    # Extract the data for this variable
      data_for_var = df[[Conditions, var, 'Repeat', 'File_name' ]]

    # Save the data_for_var to a CSV for replotting
      data_for_var.to_csv(os.path.join(Results_Folder, "csv", f"{var}_boxplot_data.csv"), index=False)

    # Calculate the Interquartile Range (IQR) using the 25th and 75th percentiles
      Q1 = df[var].quantile(0.25)
      Q3 = df[var].quantile(0.75)
      IQR = Q3 - Q1

    # Define bounds for the outliers
      multiplier = 10
      lower_bound = Q1 - multiplier * IQR
      upper_bound = Q3 + multiplier * IQR

    # Plotting
      sns.boxplot(x=Conditions, y=var, data=filtered_df, ax=ax_box, color='lightgray')  # Boxplot
      sns.stripplot(x=Conditions, y=var, data=filtered_df, ax=ax_box, hue='Repeat', dodge=True, jitter=True, alpha=0.2)  # Individual data points
      ax_box.set_ylim([max(min(filtered_df[var]), lower_bound), min(max(filtered_df[var]), upper_bound)])
      ax_box.set_title(f"{var}")
      ax_box.set_xlabel('Condition')
      ax_box.set_ylabel(var)
      tick_labels = ax_box.get_xticklabels()
      tick_locations = ax_box.get_xticks()
      ax_box.xaxis.set_major_locator(FixedLocator(tick_locations))
      ax_box.set_xticklabels(tick_labels, rotation=90)
      ax_box.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='Repeat')

    # Statistical Analyses and Heatmaps

    # Effect Size heatmap ax
      ax_d = fig.add_subplot(gs[1, 0])
      sns.heatmap(effect_size_matrices[var].fillna(0), annot=True, cmap="viridis", cbar=True, square=True, ax=ax_d, vmax=1)
      ax_d.set_title(f"Effect Size (Cohen's d) for {var}")

    # p-value heatmap ax
      ax_p = fig.add_subplot(gs[1, 1])
      sns.heatmap(p_value_matrices[var].fillna(1), annot=True, cmap="viridis_r", cbar=True, square=True, ax=ax_p, vmax=0.1)
      ax_p.set_title(f"Randomization Test p-value for {var}")

    # Bonferroni corrected p-value heatmap ax
      ax_bonf = fig.add_subplot(gs[1, 2])
      sns.heatmap(bonferroni_matrices[var].fillna(1), annot=True, cmap="viridis_r", cbar=True, square=True, ax=ax_bonf, vmax=0.1)
      ax_bonf.set_title(f"Bonferroni-corrected p-value for {var}")

      plt.tight_layout()
      pdf_pages.savefig(fig)

    # Close the PDF
      pdf_pages.close()

condition_selector = display_condition_selection(df_to_plot, Conditions)
selectable_columns = get_selectable_columns(df_to_plot)
variable_checkboxes = display_variable_checkboxes(selectable_columns)

button = Button(description="Plot Selected Variables", layout=Layout(width='400px'), button_style='info')
button.on_click(lambda b: plot_selected_vars(b, variable_checkboxes, df_to_plot, Conditions, base_folder, condition_selector))
display(button)

# **Part 7. Version log**
---
<font size = 4>While I strive to provide accurate and helpful information, please be aware that:
  - This notebook may contain bugs.
  - Features are currently limited and will be expanded in future releases.

<font size = 4>We encourage users to report any issues or suggestions for improvement. Please check the [repository](https://github.com/guijacquemet/CellTracksColab) regularly for updates and the latest version of this notebook.

#### **Known Issues**:
- Tracks are displayed in 2D in section 1.4

<font size = 4>**Version 10.0.1**
  - Includes a general data reader
    
<font size = 4>**Version 0.9.2**
  - Added the Origin normalized plots

<font size = 4>**Version 0.9.1**
  - Added the PIP freeze option to save a requirement text
  - Added the heatmap visualisation of track parameters
  - Heatmaps can now be displayed on multiple pages
  - Fix userwarning message during plotting (all box plots)
  - Added the possibility to copy and paste an existing list of selected metric for clustering analyses

<font size = 4>**Version 0.9**
  - Improved plotting strategy. Specific conditions can be chosen
  - absolute cohen d values are now shown
  - In the QC the heatmap is automatically divided in subplot when too many columns are in the df

<font size = 4>**Version 0.8**
  - Settings are now saved
  - Order of the section has been modified to help streamline biological discoveries
  - New section added to quality Control to check if the dataset is balanced
  - New section added to the UMAP and tsne section to plot track parameters for selected clusters
  - clusters for UMAP and t-sne are now saved in the dataframe separetly

<font size = 4>**Version 0.7**
  - check_for_nans function added
  - Clustering using t-SNE added

<font size = 4>**Version 0.6**
  - Improved organisation of the results
  - Tracks visualisation are now saved

<font size = 4>**Version 0.5**
  - Improved part 5
  - Added the possibility to find examplar on the raw movies when available
  - Added the possibility to export video with the examplar labeled
  - Code improved to deal with larger dataset (tested with over 50k tracks)
  - test dataset now contains raw video and is hosted on Zenodo
  - Results are now organised in folders
  - Added progress bars
  - Minor code fixes

<font size = 4>**Version 0.4**

  - Added the possibility to filter and smooth tracks
  - Added spatial and temporal calibration
  - Notebook is streamlined
  - multiple bug fix
  - Remove the t-sne
  - Improved documentation

<font size = 4>**Version 0.3**
  - Fix a nasty bug in the import functions
  - Add basic examplar for UMAP
  - Added the statistical analyses and their explanations.
  - Added a new quality control part that helps assessing the similarity of results between FOV, conditions and repeats
  - Improved part 5 (previously part 4).

<font size = 4>**Version 0.2**
  - Added support for 3D tracks
  - New documentation and metrics added.

<font size = 4>**Version 0.1**
This is the first release of this notebook.

---